INSPIRATION: https://github.com/dusanBirta/Animate-Photos/blob/main/First_Order_Motion.ipynb

In [9]:
# Clone the repository
!git clone https://github.com/AliaksandrSiarohin/first-order-model

# Navigate to the directory
%cd first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 397, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 397 (delta 38), reused 34 (delta 34), pack-reused 345 (from 3)
Receiving objects: 100% (397/397), 72.19 MiB | 26.54 MiB/s, done.
Resolving deltas: 100% (205/205), done.
/content/first-order-model/first-order-model


In [10]:
# Download the VoxCeleb model which is trained for facial animations
!pip install ffmpeg-python
!apt install ffmpeg
!pip install gdown
!gdown --id 1L8P-hpBhZi8Q_1vP2KlQ4N6dvlzpYBvZ
#!wget "https://disk.yandex.com/d/lEw8uRm140L_eQ" -O vox-cpk.pth.tar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1L8P-hpBhZi8Q_1vP2KlQ4N6dvlzpYBvZ
From (redirected): https://drive.google.com/uc?id=1L8P-hpBhZi8Q_1vP2KlQ4N6dvlzpYBvZ&confirm=t&uuid=b4e832e1-ad16-4003-8a29-bf8b303004d3
To: /content/first-order-model/first-order-model/vox-adv-cpk.pth.tar
100% 751M/751M [00:02<00:00, 256MB/s]


In [16]:
from google.colab import files
import imageio

# Upload source image
uploaded = files.upload()
source_image_path = list(uploaded.keys())[0]
source_image = imageio.imread(source_image_path)

import requests

# Download the driving video from the provided link
# url = 'https://github.com/dusanBirta/Animate-Photos/raw/main/driving.mp4'
# r = requests.get(url)


# Save it to a temporary file
# with open('../TEMP.mp4', 'wb') as f:
#     f.write(r.content)

# Now, read the saved video
# driving_video_path = 'TEMP.mp4'
uploaded_video = files.upload()
driving_video_path = list(uploaded_video.keys())[0]
reader = imageio.get_reader(driving_video_path)
driving_video = [frame for frame in reader]


Saving photo_5201865317637287374_m.jpg to photo_5201865317637287374_m (1).jpg


Saving my_face.mp4 to my_face.mp4


In [17]:
#import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import Video, display
import warnings
warnings.filterwarnings("ignore")

# Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml',
                            checkpoint_path='vox-adv-cpk.pth.tar')

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

  0%|          | 0/329 [00:00<?, ?it/s]

In [18]:
# Save directly using imageio
animation_path = source_image_path.rsplit('.', 1)[0] + "_output.mp4"
imageio.mimsave(animation_path, [img_as_ubyte(frame) for frame in predictions], fps=20)

import os
if os.path.exists(animation_path):
    print(f"File {animation_path} saved successfully!")
else:
    print(f"Failed to save {animation_path}.")

File photo_5201865317637287374_m (1)_output.mp4 saved successfully!


In [19]:
from IPython.display import HTML
from base64 import b64encode

def play_video(video_path):
    # Load the video
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    # Display the video
    return HTML(f"""
    <video width="400" controls>
        <source src="{data_url}" type="video/mp4">
    </video>
    """)

video_path = f"/content/first-order-model/first-order-model/{animation_path}"
display(play_video(video_path))